In [ ]:
import os
import pandas as pd
import numpy as np

# Define the folder containing the Excel files
folder_path = "/content/"  # Update this with the actual path
output_file = "merged_data_July2025.xlsx"

# List all Excel files in the folder
excel_files = [f for f in os.listdir(folder_path) if f.endswith(".xlsx")]

# Read the first file to get sheet names (assuming all files have the same sheets)
sample_file = os.path.join(folder_path, excel_files[0])
sheet_names = pd.ExcelFile(sample_file).sheet_names

# Add additional specific sheets if not already present
additional_sheets = ["bestMealTagCatResults"]
for sheet in additional_sheets:
    if sheet not in sheet_names:
        sheet_names.append(sheet)

# Dictionary to store merged data for each sheet
merged_data = {sheet: [] for sheet in sheet_names}

# Read and merge sheets across all files
for file in excel_files:
    file_path = os.path.join(folder_path, file)
    xls = pd.ExcelFile(file_path)

    for sheet in sheet_names:
        if sheet in xls.sheet_names:  # Check if the sheet exists in the current file
            df = pd.read_excel(xls, sheet_name=sheet)
            df["Source File"] = file  # Add a column to track source file

            # Check if relevant columns exist
            if "Meal Tag" in df.columns and "Time" in df.columns:
                # Sort by "Meal Tag" and prioritize rows where "Time" == 0
                df_sorted = df.sort_values(by=["Meal Tag", "Time"], ascending=[True, True])
                df_filtered = df_sorted.groupby("Meal Tag").first().reset_index()  # Keep one row per meal tag
            else:
                df_filtered = df  # If no Meal Tag or Time column, keep all data

            merged_data[sheet].append(df_filtered)

# Save merged data to a new Excel file
MAX_EXCEL_ROWS = 1048576

with pd.ExcelWriter(output_file) as writer:
    for sheet, dfs in merged_data.items():
        if dfs:  # Only write if there's data
            merged_df = pd.concat(dfs, ignore_index=True)

            # Split into chunks if necessary
            if len(merged_df) > MAX_EXCEL_ROWS:
                num_parts = (len(merged_df) // MAX_EXCEL_ROWS) + 1
                for i in range(num_parts):
                    start_row = i * MAX_EXCEL_ROWS
                    end_row = start_row + MAX_EXCEL_ROWS
                    chunk_df = merged_df.iloc[start_row:end_row]
                    chunk_sheet_name = f"{sheet}_Part{i+1}"
                    chunk_df.to_excel(writer, sheet_name=chunk_sheet_name, index=False)
            else:
                merged_df.to_excel(writer, sheet_name=sheet, index=False)

print(f"Merged data saved to {output_file}")




Merged data saved to merged_data_July2025.xlsx


In [2]:
file_path = "/content/merged_data_July2025.xlsx"  # Ensure this path is correct
xls = pd.ExcelFile(file_path)

print(xls.sheet_names)  # Print all available sheet names



['GlucoseEvents', 'GroupStats', 'CohesionMetricsWithANOVA', 'MealTagSim', 'best_MealTagSim', 'MealCat_Cluster_Sim', 'Best_MealCat_ClusteSim', 'Cat_Cluster_Pump', 'Best_Cat_Cluster_Pump', 'bestMealTagCatResults']


In [3]:
import pandas as pd
import re

# File path
file_path = "/content/merged_data_July2025.xlsx"

# Load Excel file
xls = pd.ExcelFile(file_path)

# Define thresholds
LOW_THRESHOLD = 3.9
HIGH_THRESHOLD = 10.0
TIGHT_UPPER = 7.8
SIM_DURATION = 240  # in minutes

# Relevant sheets (added "tags+MealCatSim")
sheets_to_compare = [
    "MealTagSim",
    "best_MealTagSim",
    "MealCat_Cluster_Sim",
    "Best_MealCat_ClusteSim",
    "Cat_Cluster_Pump",
    "Best_Cat_Cluster_Pump",
    "bestMealTagCatResults",
    "tags+MealCatSim"
]

# Column mappings
columns_to_check = {
    "Learned Glucose Response": "Learned",
    "Simulated Glucose Response": "Simulated"
}

# Summary storage
summary = []

# Process each sheet
for sheet in sheets_to_compare:
    try:
        df = pd.read_excel(file_path, sheet_name=sheet)
    except Exception as e:
        print(f"Could not read sheet {sheet}: {e}")
        continue

    for true_col_name, label in columns_to_check.items():
        if true_col_name not in df.columns:
            print(f"Column '{true_col_name}' not found in sheet {sheet}")
            continue

        print(f"\nSheet: {sheet}, Column: '{true_col_name}' - Raw Data Types:")
        print(df[true_col_name].apply(lambda x: type(x)).value_counts())

        # Clean data: remove units, symbols, and whitespace
        cleaned_series = df[true_col_name].astype(str).str.replace(r"[^\d\.]", "", regex=True).str.strip()

        # Convert to numeric
        series = pd.to_numeric(cleaned_series, errors='coerce')

        total_points = series.notnull().sum()
        if total_points == 0:
            print(f"No valid numeric data in column '{true_col_name}' of sheet {sheet}")
            continue

        # Time per data point
        time_per_point = SIM_DURATION / total_points

        # Calculate time in ranges
        tbr = (series < LOW_THRESHOLD).sum() * time_per_point
        tar = (series > HIGH_THRESHOLD).sum() * time_per_point
        titr = ((series >= LOW_THRESHOLD) & (series <= TIGHT_UPPER)).sum() * time_per_point

        summary.append({
            "Sheet": sheet,
            "Column": label,
            "Time Below Range (<3.9 mmol/L)": round(tbr, 2),
            "Time In Tight Range (3.9 - 7.8 mmol/L)": round(titr, 2),
            "Time Above Range (>10 mmol/L)": round(tar, 2),
        })

# Output summary
summary_df = pd.DataFrame(summary)
print(summary_df)

# Export to Excel
summary_df.to_excel("glucose_range_summary.xlsx", index=False)



Sheet: MealTagSim, Column: 'Learned Glucose Response' - Raw Data Types:
Learned Glucose Response
<class 'str'>    341880
Name: count, dtype: int64

Sheet: MealTagSim, Column: 'Simulated Glucose Response' - Raw Data Types:
Simulated Glucose Response
<class 'str'>    341880
Name: count, dtype: int64

Sheet: best_MealTagSim, Column: 'Learned Glucose Response' - Raw Data Types:
Learned Glucose Response
<class 'str'>    4355
Name: count, dtype: int64
No valid numeric data in column 'Learned Glucose Response' of sheet best_MealTagSim

Sheet: best_MealTagSim, Column: 'Simulated Glucose Response' - Raw Data Types:
Simulated Glucose Response
<class 'str'>    4355
Name: count, dtype: int64
No valid numeric data in column 'Simulated Glucose Response' of sheet best_MealTagSim

Sheet: MealCat_Cluster_Sim, Column: 'Learned Glucose Response' - Raw Data Types:
Learned Glucose Response
<class 'str'>    293370
Name: count, dtype: int64
No valid numeric data in column 'Learned Glucose Response' of sheet

In [4]:

# Define the correct file path
file_path = "/content/merged_data_July2025.xlsx"  # Ensure this path is correct
xls = pd.ExcelFile(file_path)

# Print available sheet names
print("Available sheets:", xls.sheet_names)

# Sheets to compare (use all available sheets)
sheets_to_compare = [
    'MealTagSim', 'best_MealTagSim', 'MealCat_Cluster_Sim',
    'Best_MealCat_ClusteSim', 'Cat_Cluster_Pump', 'Best_Cat_Cluster_Pump',
    'bestMealTagCatResults'
]

# Load sheets
dfs = {sheet: pd.read_excel(file_path, sheet_name=sheet) for sheet in sheets_to_compare}

# Variable to store the unique learned value
learned_time_in_range = None

# Process each sheet
for sheet in sheets_to_compare:
    df = dfs[sheet]
    print(f"DEBUG - Columns in '{sheet}': {df.columns.tolist()}")

    # Standardize column names
    df.columns = [col.strip() for col in df.columns]

    # Identify columns dynamically
    best_col, learned_col = None, None
    for col in df.columns:
        col_lower = col.lower().strip()
        if "time in range" in col_lower:
            best_col = col  # Assume this as the "best" metric
        if "learned glucose response" in col_lower:
            learned_col = col  # Identify the learned metric

    print(f"DEBUG - {sheet}: Best Column -> {best_col}, Learned Column -> {learned_col}")

    # Raise an error if the "Best Time in Range" column is missing
    if best_col is None:
        raise ValueError(f"ERROR: Sheet '{sheet}' is missing required column: 'Best Time in Range (%)'")

    # Extract the single "Learned Time in Range (%)" value if available
    if learned_col and learned_time_in_range is None:
        try:
            df[learned_col] = df[learned_col].apply(lambda x: np.mean(eval(x)) if isinstance(x, str) and x.startswith("[") else x)
            learned_time_in_range = df[learned_col].astype(float).mean()
        except Exception as e:
            print(f"ERROR: Failed to compute mean for '{learned_col}' in sheet '{sheet}': {e}")
            learned_time_in_range = None

    # Aggregate "Best Time in Range (%)" by "Source File"
    dfs[sheet] = df.groupby("Source File", as_index=False).agg({best_col: "mean"})

    # Rename column uniquely
    dfs[sheet].rename(columns={best_col: f"Best Time in Range (%)_{sheet}"}, inplace=True)

# Debug - Print Columns Before Merge
for sheet in sheets_to_compare:
    print(f"DEBUG - {sheet} final columns before merge: {dfs[sheet].columns.tolist()}")

# Merge data using `best_MealTagSim` as the base
merged_df = dfs[sheets_to_compare[0]]

for sheet in sheets_to_compare[1:]:
    merged_df = merged_df.merge(
        dfs[sheet][["Source File", f"Best Time in Range (%)_{sheet}"]],
        on="Source File",
        how="left"
    )

# Add the single "Learned Time in Range (%)" value to all rows
if learned_time_in_range is not None:
    merged_df["Learned Time in Range (%)"] = learned_time_in_range
else:
    print("WARNING: No 'Learned Time in Range (%)' column was found!")

print("DEBUG - Final merged columns:", merged_df.columns.tolist())

# Check for missing values
for sheet in sheets_to_compare:
    col_name = f"Best Time in Range (%)_{sheet}"
    if merged_df[col_name].isnull().sum() > 0:
        print(f"WARNING: Missing values detected in {col_name} after merge!")

# Split into chunks if necessary
max_rows = 1_000_000
chunks = [merged_df[i:i + max_rows] for i in range(0, len(merged_df), max_rows)]

# Save results
output_file = "time_in_range_analysis.xlsx"
with pd.ExcelWriter(output_file) as writer:
    for i, chunk in enumerate(chunks):
        chunk.to_excel(writer, sheet_name=f"Analysis_Part{i+1}", index=False)

print(f"Analysis saved to {output_file}")


Available sheets: ['GlucoseEvents', 'GroupStats', 'CohesionMetricsWithANOVA', 'MealTagSim', 'best_MealTagSim', 'MealCat_Cluster_Sim', 'Best_MealCat_ClusteSim', 'Cat_Cluster_Pump', 'Best_Cat_Cluster_Pump', 'bestMealTagCatResults']
DEBUG - Columns in 'MealTagSim': ['Unnamed: 0', 'MealTag', 'Insulin Doses', 'Bolus Offsets', 'Total Insulin Dose', 'Time in Range (%)', 'Learned Time in Range (%)', 'Simulated Glucose Response', 'Learned Glucose Response', 'Source File']
DEBUG - MealTagSim: Best Column -> Learned Time in Range (%), Learned Column -> Learned Glucose Response
DEBUG - Columns in 'best_MealTagSim': ['Unnamed: 0', 'MealTag', 'Insulin Doses', 'Bolus Offsets', 'Total Insulin Dose', 'Time in Range (%)', 'Learned Time in Range (%)', 'Simulated Glucose Response', 'Learned Glucose Response', 'Source File']
DEBUG - best_MealTagSim: Best Column -> Learned Time in Range (%), Learned Column -> Learned Glucose Response
DEBUG - Columns in 'MealCat_Cluster_Sim': ['Unnamed: 0', 'Meal Category', 

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
# Directory to save the plots
save_dir = 'plots/'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

# Load the sheet by name
MealCat_df = xls.parse('Best_MealCat_ClusteSim')

# Strip column names of extra spaces
MealCat_df.columns = MealCat_df.columns.str.strip()
print("Column names:", MealCat_df.columns.tolist())  # Debug

# Check head of the dataframe
print("First few rows:\n", MealCat_df.head())  # Debug

# Drop rows with missing required columns
MealCat_df = MealCat_df.dropna(subset=['Time in Range (%)', 'Learned Time in Range (%)', 'Meal Category'])

# Calculate the difference between Time in Range and Learned Time in Range
MealCat_df['Diff in Time in Range (%)'] = MealCat_df['Time in Range (%)'] - MealCat_df['Learned Time in Range (%)']

# Categorize Learned Time in Range (TIR)
MealCat_df['TIR Category'] = MealCat_df['Learned Time in Range (%)'].apply(lambda x: '>70%' if x > 70 else '<70%')

# Normalize meal categories (lowercase and strip)
MealCat_df['Meal Category'] = MealCat_df['Meal Category'].str.strip().str.lower()

# Define and enforce the specific order of meal categories
meal_order = ['breakfast', 'lunch', 'dinner', 'snack']
MealCat_df = MealCat_df[MealCat_df['Meal Category'].isin(meal_order)]

print(f"Filtered data has {len(MealCat_df)} rows.")  # Debug

if MealCat_df.empty:
    print("No data available for the specified meal categories.")
else:
    # Set up grayscale palette
    gray_palette_1 = ['#d9d9d9', '#bdbdbd', '#969696', '#636363']  # Meal categories
    gray_palette_2 = ['#bdbdbd', '#636363']  # TIR categories

    # Plot 1 and 2 side-by-side
    plt.figure(figsize=(12, 6))

    # Boxplot 1: TIR difference by Meal Category
    plt.subplot(1, 2, 1)
    sns.boxplot(x='Meal Category', y='Diff in Time in Range (%)', data=MealCat_df,
                order=meal_order, palette=gray_palette_1)
    plt.xticks(rotation=45)
    plt.title('Distribution of TIR Difference by Meal Category')
    plt.xlabel('Meal Category')
    plt.ylabel('Difference in Time in Range (%)')

    # Boxplot 2: TIR difference by TIR Category
    plt.subplot(1, 2, 2)
    sns.boxplot(x='TIR Category', y='Diff in Time in Range (%)', data=MealCat_df,
                palette=gray_palette_2)
    plt.title('Distribution of TIR Difference by Learned TIR (>70% vs. <70%)')
    plt.xlabel('Learned Time in Range Category')
    plt.ylabel('Difference in Time in Range (%)')

    # Save first figure
    plt.tight_layout()
    output_path1 = os.path.join(save_dir, "Meal_Cat_TIR_Boxplots.png")
    plt.savefig(output_path1, dpi=300, bbox_inches='tight')
    plt.close()  # Close the plot
    print(f"Plot saved as: {output_path1}")

    # 🎯 New Plot: Meal Category TIR Differences by Participant
    participant_order = ['UoM2301', 'UoM2302', 'UoM2304', 'UoM2304', 'UoM2305', 'UoM2306',
                         'UoM2307', 'UoM2308', 'UoM2309', 'UoM2313', 'UoM2314', 'UoM2320',
                         'UoM2403', 'UoM2404', 'UoM2405']  # Specified order

    # Check if 'Source File' or 'Participant ID' column exists and assign to 'Participant'
    if 'Source File' in MealCat_df.columns:
        MealCat_df['Participant'] = MealCat_df['Source File'].str.replace('.xlsx', '', regex=True)
    elif 'Participant ID' in MealCat_df.columns:
        MealCat_df['Participant'] = MealCat_df['Participant ID']
    else:
        raise ValueError("No column found for participant identification (e.g., 'Source File').")

    plt.figure(figsize=(14, 6))
    sns.boxplot(x='Participant', y='Diff in Time in Range (%)',
                hue='Meal Category', data=MealCat_df, hue_order=meal_order,
                palette=gray_palette_1, order=participant_order)  # Ordering participants

    plt.xticks(rotation=45)
    plt.title('TIR Difference by Participant and Meal Category')
    plt.xlabel('Participant')
    plt.ylabel('Difference in Time in Range (%)')
    plt.legend(title='Meal Category', bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.tight_layout()

    output_path2 = os.path.join(save_dir, "Meal_Cat_TIR_by_Participant.png")
    plt.savefig(output_path2, dpi=300, bbox_inches='tight')
    plt.close()  # Close the plot
    print(f"Additional plot saved as: {output_path2}")

In [ ]:
import pandas as pd
from scipy import stats
import numpy as np

# Load the sheet by name
MealCat_df = xls.parse('Best_MealCat_ClusteSim')

# Strip column names of extra spaces
MealCat_df.columns = MealCat_df.columns.str.strip()

# Fix: Calculate TIR improvement as Learned - Baseline
MealCat_df['TIR_Improvement'] = MealCat_df['Time in Range (%)'] - MealCat_df['Learned Time in Range (%)']

# Show individual-level data for reporting
print("\n📌 Individual Participant TIR Data:")
print(MealCat_df[['Source File', 'Meal Category', 'Cluster', 'Time in Range (%)', 'Learned Time in Range (%)', 'TIR_Improvement']])

# Split into groups based on Learned TIR
above_70 = MealCat_df[MealCat_df['Learned Time in Range (%)'] > 70]
below_70 = MealCat_df[MealCat_df['Learned Time in Range (%)'] <= 70]

# Define a function for clean summary stats
def summarize(group, label):
    return {
        'Group': label,
        'N': group.shape[0],
        'Mean Improvement': group['TIR_Improvement'].mean(),
        'Std Dev': group['TIR_Improvement'].std(),
        'Median': group['TIR_Improvement'].median(),
        'Min': group['TIR_Improvement'].min(),
        'Max': group['TIR_Improvement'].max()
    }

# Get summary for each group
summary_data = [summarize(above_70, 'Learned TIR > 70%'), summarize(below_70, 'Learned TIR ≤ 70%')]
summary_df = pd.DataFrame(summary_data)

# Display summary
print("\n📊 Summary Statistics for TIR Improvement by Group:")
print(summary_df)

# 🔢 Optional: Perform Independent T-Test (assumes normality)
t_stat, p_value = stats.ttest_ind(above_70['TIR_Improvement'], below_70['TIR_Improvement'], equal_var=False)

# 🧠 Optional: Calculate Cohen’s d
def cohens_d(x, y):
    nx = len(x)
    ny = len(y)
    pooled_std = np.sqrt(((nx - 1)*np.var(x, ddof=1) + (ny - 1)*np.var(y, ddof=1)) / (nx + ny - 2))
    return (np.mean(x) - np.mean(y)) / pooled_std

cohen_d = cohens_d(above_70['TIR_Improvement'], below_70['TIR_Improvement'])

print("\n🧪 Statistical Test:")
print(f"T-test: t = {t_stat:.3f}, p = {p_value:.4f}")
print(f"Cohen's d: {cohen_d:.3f} (effect size)")


In [ ]:
# 📊 Detailed per-meal summary by Learned TIR group
def detailed_meal_summary(df, group_label):
    print(f"\n🍽️ Meal-specific TIR for group: {group_label}")
    meal_groups = df.groupby('Meal Category')

    for meal, group in meal_groups:
        learned_mean = group['Learned Time in Range (%)'].mean()
        learned_std = group['Learned Time in Range (%)'].std()
        sim_mean = group['Time in Range (%)'].mean()
        sim_std = group['Time in Range (%)'].std()

        print(f"\nFor those where the TIR was {group_label} for {meal.lower()}, "
              f"learned TIR averaged {learned_mean:.1f}% +/- {learned_std:.1f}%, "
              f"using the simulation the TIR increased to {sim_mean:.1f}% +/- {sim_std:.1f}%.")

# Apply the summaries for both groups
detailed_meal_summary(above_70, '>70%')
detailed_meal_summary(below_70, '≤70%')


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import os

# Define the directory to save plots
save_dir = 'plots/'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

# Load Excel file
tag_df = xls.parse('best_MealTagSim')  # Load the first sheet

# Clean column names (remove spaces)
tag_df.columns = tag_df.columns.str.strip()

# Remove ".xlsx" from 'Source File' column
tag_df['Source File'] = tag_df['Source File'].str.replace('.xlsx', '', regex=True)

# Calculate the difference between Time in Range and Learned Time in Range
tag_df['Time in range difference (%)'] = tag_df['Time in Range (%)'] - tag_df['Learned Time in Range (%)']

# Categorize Learned Time in Range (TIR)
tag_df['TIR Category'] = tag_df['Learned Time in Range (%)'].apply(lambda x: '>70%' if x > 70 else '<70%')

# Define grayscale colors for TIR categories
gray_palette = {
    '>70%': '#636363',  # Dark gray
    '<70%': '#bdbdbd'   # Light gray
}

# Define the order for participants
participant_order = ['UoM2301', 'UoM2302', 'UoM2304', 'UoM2305', 'UoM2306',
                     'UoM2307', 'UoM2308', 'UoM2309', 'UoM2313', 'UoM2314',
                     'UoM2320', 'UoM2403', 'UoM2404', 'UoM2405']

# Set up the figure
plt.figure(figsize=(12, 6))

# Create the box plot
sns.boxplot(x='Source File', y='Time in range difference (%)', hue='TIR Category',
            data=tag_df, palette=gray_palette, order=participant_order)

# Customize appearance
plt.xticks(rotation=45)
plt.title('Distribution of the Meal Tag Simulation Time in Range Difference by Participant and TIR Category')
plt.xlabel('Participant ID', fontweight='bold')
plt.ylabel('Time in Range Difference (%)', fontweight='bold')
plt.legend(title='Learned Time in Range')

# Remove border (top and right spines)
sns.despine()

# Save the plot
output_path = os.path.join(save_dir, 'Tag_TIR_Difference_by_Participant.png')
plt.savefig(output_path, dpi=300, bbox_inches='tight')

# Show the plot
plt.show()

print(f"Plot saved as: {output_path}")


In [ ]:
import pandas as pd
from scipy import stats
import numpy as np

# Load the correct sheet by name
MealTag_df = xls.parse('best_MealTagSim')  # <--- changed sheet name here

# Strip column names of extra spaces
MealTag_df.columns = MealTag_df.columns.str.strip()

print("🧾 Column Names in the Sheet:")
print(MealTag_df.columns.tolist())


# Fix: Calculate TIR improvement as Learned - Baseline (positive = improvement)
MealTag_df['TIR_Improvement'] = MealTag_df['Time in Range (%)'] - MealTag_df['Learned Time in Range (%)']

# Show individual-level data for reporting
print("\n📌 Individual Participant TIR Data:")
print(MealTag_df[['Source File', 'MealTag', 'Time in Range (%)', 'Learned Time in Range (%)', 'TIR_Improvement']])

# Split into groups based on Learned TIR
above_70 = MealTag_df[MealTag_df['Learned Time in Range (%)'] > 70]
below_70 = MealTag_df[MealTag_df['Learned Time in Range (%)'] <= 70]

# Define a function for clean summary stats
def summarize(group, label):
    return {
        'Group': label,
        'N': group.shape[0],
        'Mean Improvement': group['TIR_Improvement'].mean(),
        'Std Dev': group['TIR_Improvement'].std(),
        'Median': group['TIR_Improvement'].median(),
        'Min': group['TIR_Improvement'].min(),
        'Max': group['TIR_Improvement'].max()
    }

# Get summary for each group
summary_data = [summarize(above_70, 'Learned TIR > 70%'), summarize(below_70, 'Learned TIR ≤ 70%')]
summary_df = pd.DataFrame(summary_data)

# Display summary
print("\n📊 Summary Statistics for TIR Improvement by Group:")
print(summary_df)

# 🔢 Perform Independent T-Test
t_stat, p_value = stats.ttest_ind(
    above_70['TIR_Improvement'],
    below_70['TIR_Improvement'],
    equal_var=False
)

# 🧠 Calculate Cohen’s d
def cohens_d(x, y):
    nx = len(x)
    ny = len(y)
    pooled_std = np.sqrt(((nx - 1)*np.var(x, ddof=1) + (ny - 1)*np.var(y, ddof=1)) / (nx + ny - 2))
    return (np.mean(x) - np.mean(y)) / pooled_std

cohen_d = cohens_d(above_70['TIR_Improvement'], below_70['TIR_Improvement'])

# Print results
print("\n🧪 Statistical Test:")
print(f"T-test: t = {t_stat:.3f}, p = {p_value:.4f}")
print(f"Cohen's d: {cohen_d:.3f} (effect size)")



In [ ]:


# 📊 Detailed per-mealTag summary by Learned TIR group
def detailed_mealtag_summary(df, group_label):
    print(f"\n🍱 MealTag-specific TIR for group: {group_label}")
    tag_groups = df.groupby('MealTag')

    for tag, group in tag_groups:
        learned_mean = group['Learned Time in Range (%)'].mean()
        learned_std = group['Learned Time in Range (%)'].std()
        sim_mean = group['Time in Range (%)'].mean()
        sim_std = group['Time in Range (%)'].std()

        print(f"\nFor those where the TIR was {group_label} for '{tag.lower()}', "
              f"learned TIR averaged {learned_mean:.1f}% +/- {learned_std:.1f}%, "
              f"using the simulation the TIR increased to {sim_mean:.1f}% +/- {sim_std:.1f}%.")

# Apply the summaries for both TIR groups
detailed_mealtag_summary(above_70, '>70%')
detailed_mealtag_summary(below_70, '≤70%')


In [ ]:
# Directory to save the plots
save_dir = 'plots/'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

# Load the correct sheet by name
MealTag_df = xls.parse('bestMealTagCatResults')  # <-- Updated sheet name

# Strip column names of extra spaces
MealTag_df.columns = MealTag_df.columns.str.strip()

# Remove ".xlsx" from 'Source File' column
MealTag_df['Source File'] = MealTag_df['Source File'].str.replace('.xlsx', '', regex=True)

# Define the order for participants
participant_order = ['UoM2301', 'UoM2302', 'UoM2304', 'UoM2305', 'UoM2306',
                     'UoM2307', 'UoM2308', 'UoM2309', 'UoM2313', 'UoM2314',
                     'UoM2320','UoM2403', 'UoM2404', 'UoM2405']  # Specify order

print("🧾 Column Names in the Sheet:")
print(MealTag_df.columns.tolist())

# Fix: Calculate TIR improvement as Learned - Baseline
MealTag_df['TIR_Improvement'] = MealTag_df['Time in Range (%)'] - MealTag_df['Learned Time in Range (%)']

# 🧩 Overall Improvement
overall_mean = MealTag_df['TIR_Improvement'].mean()
print(f"\n🌍 Overall Mean TIR Improvement: {overall_mean:.2f}%")

# 👤 Improvement by Participant (Source File)
by_participant = MealTag_df.groupby('Source File')['TIR_Improvement'].agg(['mean', 'std', 'count']).reset_index()
print("\n👤 TIR Improvement by Participant:")
print(by_participant)

# 🍽️ Improvement by Meal Category
by_meal_category = MealTag_df.groupby('Meal Category')['TIR_Improvement'].agg(['mean', 'std', 'count']).reset_index()
print("\n🍽️ TIR Improvement by Meal Category:")
print(by_meal_category)

# 🔢 Improvement by Cluster Number
by_cluster = MealTag_df.groupby('Cluster')['TIR_Improvement'].agg(['mean', 'std', 'count']).reset_index()
print("\n🔢 TIR Improvement by Cluster:")
print(by_cluster)

# Set plot style
sns.set(style="whitegrid")

# Filter for specific meal categories
desired_meals = ['breakfast', 'lunch', 'dinner', 'snacks']
filtered_df = MealTag_df[MealTag_df['Meal Category'].str.lower().isin(desired_meals)]

# 📊 Plot: Improvement by Meal Category
plt.figure(figsize=(10, 5))
sns.barplot(x='Meal Category', y='TIR_Improvement', data=filtered_df, ci='sd', palette='Greys')
plt.title('TIR Improvement by Meal Category')
plt.ylabel('Mean TIR Improvement (%)')
plt.xlabel('Meal Category')
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig(os.path.join(save_dir, 'tir_improvement_by_selected_meals.png'), dpi=300)  # Save plot
plt.close()  # Close the plot

# 📊 Plot: Improvement by Cluster
plt.figure(figsize=(8, 5))
sns.barplot(x='Cluster', y='TIR_Improvement', data=MealTag_df, ci='sd', palette='Greys')
plt.title('TIR Improvement by Cluster')
plt.ylabel('Mean TIR Improvement (%)')
plt.xlabel('Cluster Number')
plt.tight_layout()
plt.savefig(os.path.join(save_dir, 'tir_improvement_by_cluster.png'), dpi=300)  # Save plot
plt.close()  # Close the plot

# 📊 Plot: Improvement by Meal Category and Source File
plt.figure(figsize=(12, 6))
sns.barplot(x='Meal Category', y='TIR_Improvement', hue='Source File', data=filtered_df, ci='sd', palette='Greys')
plt.title('TIR Improvement by Meal Category and Source File')
plt.ylabel('Mean TIR Improvement (%)')
plt.xlabel('Meal Category')
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig(os.path.join(save_dir, 'tir_improvement_by_meal_category_and_source_file.png'), dpi=300)  # Save plot
plt.close()  # Close the plot

# 📊 Plot: Improvement by Source File
plt.figure(figsize=(12, 6))
sns.barplot(x='Source File', y='TIR_Improvement', data=MealTag_df, ci='sd', palette='Greys')
plt.title('TIR Improvement by Source File')
plt.ylabel('Mean TIR Improvement (%)')
plt.xlabel('Source File')
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig(os.path.join(save_dir, 'tir_improvement_by_source_file.png'), dpi=300)  # Save plot
plt.close()  # Close the plot

# 📊 Plot: Improvement by Participant
plt.figure(figsize=(12, 6))
sns.barplot(x='Source File', y='TIR_Improvement', hue='Source File', data=MealTag_df, ci='sd', palette='Greys')
plt.title('TIR Improvement by Participant')
plt.ylabel('Mean TIR Improvement (%)')
plt.xlabel('Source File')
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig(os.path.join(save_dir, 'tir_improvement_by_participant.png'), dpi=300)  # Save plot
plt.close()  # Close the plot


In [10]:
# Load the correct sheet by name
MealTagCat_df = xls.parse('bestMealTagCatResults')  # <-- Updated sheet name

print(MealTagCat_df.head())

# Strip whitespace from column names
MealTagCat_df.columns = MealTagCat_df.columns.str.strip()

# Remove '_MealTagCatSim.xlsx' from 'Source File' and extract participant ID
MealTagCat_df['Participant'] = MealTagCat_df['Source File'].str.extract(r'(UoM\d{4})')

# Clean up 'Source File' column to just the Participant ID (optional but keeps things tidy)
MealTagCat_df['Source File'] = MealTagCat_df['Source File'].str.extract(r'(UoM\d{4})')

# Sort participants in the custom order
participant_order = [
    'UoM2301', 'UoM2302', 'UoM2304', 'UoM2305', 'UoM2306',
    'UoM2307', 'UoM2308', 'UoM2309', 'UoM2313', 'UoM2314',
    'UoM2320', 'UoM2403', 'UoM2404', 'UoM2405'
]

MealTagCat_df['Participant'] = pd.Categorical(MealTagCat_df['Participant'], categories=participant_order, ordered=True)
MealTagCat_df['Source File'] = pd.Categorical(MealTagCat_df['Source File'], categories=participant_order, ordered=True)

# --- Calculate Improvement in Time in Range ---
# Assuming 'Time in Range (%)' contains multiple measurements, calculate improvement by subtracting the baseline value (first value per participant)
MealTagCat_df['Baseline Time in Range (%)'] = MealTagCat_df.groupby('Participant')['Time in Range (%)'].transform('first')
MealTagCat_df['Improvement in Time in Range (%)'] = MealTagCat_df['Time in Range (%)'] - MealTagCat_df['Baseline Time in Range (%)']

# Directory to save the plots
save_dir = 'plots/'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

# --- 1. Improvement in Time in Range (%) by Participant ---
plt.figure(figsize=(10, 6))
sns.boxplot(x='Participant', y='Improvement in Time in Range (%)', data=MealTagCat_df)
plt.title('Improvement in Time in Range (%) by Participant')
plt.ylabel('Improvement in Time in Range (%)')
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig(os.path.join(save_dir, 'improvement_by_participant.png'))  # Save the plot as PNG
plt.close()  # Close the plot to avoid display

# --- 2. Improvement in Time in Range (%) by Participant and Source File ---
plt.figure(figsize=(12, 6))
sns.boxplot(x='Source File', y='Improvement in Time in Range (%)', hue='Participant', data=MealTagCat_df)
plt.title('Improvement in Time in Range (%) by Source File and Participant')
plt.ylabel('Improvement in Time in Range (%)')
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig(os.path.join(save_dir, 'improvement_by_source_file.png'))  # Save the plot as PNG
plt.close()

# Define the correct order of meal categories
meal_category_order = ['Breakfast', 'Lunch', 'Dinner', 'Snacks']

# Set the 'Meal Category' column as a categorical variable with the defined order
MealTagCat_df['Meal Category'] = pd.Categorical(MealTagCat_df['Meal Category'], categories=meal_category_order, ordered=True)

# --- 3. Improvement in Time in Range (%) by Participant, Source File, and Meal Category ---
plt.figure(figsize=(14, 6))
sns.boxplot(x='Meal Category', y='Improvement in Time in Range (%)', hue='Participant', data=MealTagCat_df)
plt.title('Improvement in Time in Range (%) by Meal Category and Participant')
plt.ylabel('Improvement in Time in Range (%)')
plt.legend(title='Participant')
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig(os.path.join(save_dir, 'improvement_by_meal_category.png'))  # Save the plot as PNG
plt.close()

# Optional: If you want even more granularity
plt.figure(figsize=(16, 6))
sns.catplot(x='Meal Category', y='Improvement in Time in Range (%)', col='Participant',
            hue='Source File', kind='box', data=MealTagCat_df, height=5, aspect=1)
plt.subplots_adjust(top=0.9)
plt.suptitle('Improvement in Time in Range (%) by Meal Category, Source File, and Participant')
plt.savefig(os.path.join(save_dir, 'improvement_by_meal_category_participant.png'))  # Save the plot as PNG
plt.close()



   Unnamed: 0            MealTag Meal Category  Cluster Insulin Doses  \
0           0  ApplePie+IceCream         Snack        1      (10, 12)   
1           1        BatteredCod        Brunch        1        (2, 2)   
2           2        BatteredCod        Supper        1        (2, 2)   
3           3         Beef Pasty         Lunch        1        (2, 2)   
4           4          BeefPatty        Dinner        2        (2, 2)   

  Bolus Offsets                         Simulated Glucose Response  \
0      (-30, 0)  [6.4, 6.43, 7.08, 7.89, 8.39, 9.95, 9.4, 8.62,...   
1    (-30, -15)  [12.0, 12.04, 12.02, 10.86, 10.87, 11.26, 12.7...   
2    (-30, -15)  [15.2, 15.21, 15.27, 15.47, 15.89, 15.99, 16.1...   
3    (-30, -15)  [14.6, 14.74, 14.82, 15.06, 15.53, 15.45, 15.3...   
4    (-30, -15)  [13.8, 14.43, 14.1, 14.03, 14.36, 14.42, 15.68...   

   Time in Range (%)                           Learned Glucose Response  \
0              20.83  [6.4, 6.415457245316276, 6.7178801678892395

/tmp/ipython-input-10-1139356149.py:27: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  MealTagCat_df['Baseline Time in Range (%)'] = MealTagCat_df.groupby('Participant')['Time in Range (%)'].transform('first')


NameError: name 'plt' is not defined

In [ ]:
import seaborn as sns

# Load Excel file
baseline_df = xls.parse('GlucoseEvents')

baseline_df['Source File'] = baseline_df['Source File'].str.replace('.xlsx', '', regex=True)

# Clean column names (remove spaces)
baseline_df.columns = baseline_df.columns.str.strip()

# Melt the data for grouped plotting
baseline_df_melted = baseline_df.melt(
    id_vars=['MealCategory', 'Source File'],
    value_vars=[
        'Percentage Time in Target Range',
        'Percentage Time in Tight Range',
        'Percentage Time Below Range',
        'Percentage Time Above Range'
    ],
    var_name='Metric',
    value_name='MeanPercentage'
)

# Define orderings for clarity and consistency
meal_order = sorted(baseline_df['MealCategory'].dropna().unique())
metric_order = [
    'Percentage Time in Target Range',
    'Percentage Time in Tight Range',
    'Percentage Time Below Range',
    'Percentage Time Above Range'
]

# Create a FacetGrid with better layout
g = sns.FacetGrid(
    baseline_df_melted,
    col='Source File',
    col_wrap=3,  # fewer columns to make plots larger
    height=5,
    aspect=1.3,
    sharey=False  # allow y-axis to scale per participant for better contrast
)

# Map the barplot using the updated `errorbar` param
g.map(
    sns.barplot,
    'MealCategory',
    'MeanPercentage',
    'Metric',
    order=meal_order,
    hue_order=metric_order,
    errorbar=None,
    palette='Set2'
)

# Improve readability and layout
g.set_titles(col_template="Participant: {col_name}")
g.set_axis_labels("Meal Category", "Mean Percentage")
g.set_xticklabels(rotation=45, ha='right')
g.add_legend(title='Metric', bbox_to_anchor=(1.02, 1), loc='upper left')
plt.subplots_adjust(top=0.9)  # Make room for the suptitle
g.fig.suptitle('Glucose Metrics per Meal Category by Participant', fontsize=18)

# Save high-resolution image
output_path = "/content/Meal_Category_Metrics_by_Participant_Improved.png"
plt.savefig(output_path, dpi=300, bbox_inches='tight')

# Show the plot
plt.show()

print(f"Plot saved as: {output_path}")


For thoe >70% vs those <70% TIR

In [5]:
import ast

# Define participant groups
group_A_ids = ['UoM2302', 'UoM2304', 'UoM2306', 'UoM2308', 'UoM2320', 'UoM2404']
group_B_ids = ['Uom2301', 'UoM2305', 'UoM2307', 'UoM2309', 'UoM2313', 'UoM2314', 'UoM2403', 'UoM2405']

# Create filtered DataFrames for each group
# Meal Category simulation:
MealCatGroup_A_df = MealCat_df[MealCat_df['Source File'].str.contains('|'.join(group_A_ids))]
MealCatGroup_B_df = MealCat_df[MealCat_df['Source File'].str.contains('|'.join(group_B_ids))]

# Meal Tag Simulation:
TagGroup_A_df = MealTag_df[MealTag_df['Source File'].str.contains('|'.join(group_A_ids))]
TagGroup_B_df = MealTag_df[MealTag_df['Source File'].str.contains('|'.join(group_B_ids))]

#  Meal Tag and Cluster Simulation:
MealTagCatGroup_A_df = MealTagCat_df[MealTagCat_df['Source File'].str.contains('|'.join(group_A_ids))]
MealTagCatGroup_B_df = MealTagCat_df[MealTagCat_df['Source File'].str.contains('|'.join(group_B_ids))]

def detailed_meal_summary(df, group_name, simulation_name):
    print(f"\n📘 Simulation: {simulation_name}")
    print(f"📍 Participant Group: {group_name}")

    # Helper: calculate time in glucose ranges
    def calculate_time_in_ranges(glucose_list):
        if isinstance(glucose_list, str):
            glucose_list = ast.literal_eval(glucose_list)
        total = len(glucose_list)
        if total == 0:
            return (0, 0, 0, 0)
        below = sum(g < 3.9 for g in glucose_list) / total * 100
        tight = sum(3.9 <= g <= 7.8 for g in glucose_list) / total * 100
        target = sum(3.9 <= g <= 10 for g in glucose_list) / total * 100
        above = sum(g > 10 for g in glucose_list) / total * 100
        return below, tight, target, above

    # Split by TIR
    above_70 = df[df['Learned Time in Range (%)'] > 70]
    below_70 = df[df['Learned Time in Range (%)'] <= 70]

    def print_summary(subset, label):
        meal_groups = subset.groupby('Meal Category')
        for meal, group in meal_groups:
            learned_mean = group['Learned Time in Range (%)'].mean()
            learned_std = group['Learned Time in Range (%)'].std()
            sim_mean = group['Time in Range (%)'].mean()
            sim_std = group['Time in Range (%)'].std()

            # Apply glucose analysis
            learned_ranges = group['Learned Glucose Response'].apply(calculate_time_in_ranges)
            sim_ranges = group['Simulated Glucose Response'].apply(calculate_time_in_ranges)

            # Averages
            learned_below = learned_ranges.apply(lambda x: x[0]).mean()
            learned_tight = learned_ranges.apply(lambda x: x[1]).mean()
            learned_target = learned_ranges.apply(lambda x: x[2]).mean()
            learned_above = learned_ranges.apply(lambda x: x[3]).mean()

            sim_below = sim_ranges.apply(lambda x: x[0]).mean()
            sim_tight = sim_ranges.apply(lambda x: x[1]).mean()
            sim_target = sim_ranges.apply(lambda x: x[2]).mean()
            sim_above = sim_ranges.apply(lambda x: x[3]).mean()

            print(f"\n📊 {meal.lower()} - TIR {label}:")
            print(f"   • Learned TIR: {learned_mean:.1f}% ± {learned_std:.1f}%")
            print(f"   • Simulated TIR: {sim_mean:.1f}% ± {sim_std:.1f}%")
            print(f"   • Learned Ranges - Below: {learned_below:.1f}%, Tight: {learned_tight:.1f}%, Target: {learned_target:.1f}%, Above: {learned_above:.1f}%")
            print(f"   • Simulated Ranges - Below: {sim_below:.1f}%, Tight: {sim_tight:.1f}%, Target: {sim_target:.1f}%, Above: {sim_above:.1f}%")

    print_summary(above_70, '>70%')
    print_summary(below_70, '≤70%')




NameError: name 'MealCat_df' is not defined

In [ ]:
# Run analysis for all your data segments
# Meal Category simulation:
detailed_meal_summary(MealCatGroup_A_df, "Group A", "Meal Category Simulation")
detailed_meal_summary(MealCatGroup_B_df, "Group B", "Meal Category Simulation")

# Meal Tag simulation:
detailed_meal_summary(TagGroup_A_df, "Group A", "Meal Tag Simulation")
detailed_meal_summary(TagGroup_B_df, "Group B", "Meal Tag Simulation")

# Meal Tag and Cluster simulation:
detailed_meal_summary(MealTagCatGroup_A_df, "Group A", "Meal Tag + Cluster Simulation")
detailed_meal_summary(MealTagCatGroup_B_df, "Group B", "Meal Tag + Cluster Simulation")



In [6]:
import pandas as pd
import re
import ast


# Load the data
file_path = '/content/merged_data_July2025.xlsx'
xls = pd.ExcelFile(file_path)
df = pd.read_excel(xls, sheet_name='GlucoseEvents')

# Create the 'Participant' column by extracting the source file name without the '.xlsx' extension
df['Participant'] = df['Source File'].str.replace(r'(_MealTagCatSim)?\.xlsx$', '', regex=True)


# Display the first few rows of the dataframe to understand its structure
# print(df.head())


# Calculate the average percentage time in each range for each participant
participant_avg_times = df.groupby('Participant')[['Percentage Time in Target Range',
                                                  'Percentage Time in Tight Range',
                                                  'Percentage Time Below Range',
                                                  'Percentage Time Above Range']].mean()

# Display the results
print(participant_avg_times)



             Percentage Time in Target Range  Percentage Time in Tight Range  \
Participant                                                                    
UoM2301                            60.326087                       32.518116   
UoM2302                            89.406780                       63.088512   
UoM2304                            91.003788                       62.794613   
UoM2305                            48.394358                       27.646058   
UoM2306                            83.231751                       64.409884   
UoM2307                            53.562663                       29.355668   
UoM2308                            77.299223                       56.250000   
UoM2309                            36.245520                       18.727599   
UoM2313                            52.321105                       29.975484   
UoM2314                            63.992632                       38.967226   
UoM2320                            93.73

In [9]:

# File and threshold setup
file_path = '/content/merged_data_July2025.xlsx'
xls = pd.ExcelFile(file_path)

assessment_measures = {
    'best_MealTagSim': 'best_MealTagSim',
    'Best_MealCat_ClusteSim': 'Best_MealCat_ClusteSim',
    'Best_Cat_Cluster_Pump': 'Best_Cat_Cluster_Pump',
    'bestMealTagCatResults': 'bestMealTagCatResults'
}

tight_range_min = 3.9
tight_range_max = 7.8
below_range = 3.9
above_range = 10.0

def calculate_ranges(glucose_list):
    total = len(glucose_list)
    tight = len([g for g in glucose_list if tight_range_min <= g <= tight_range_max])
    below = len([g for g in glucose_list if g < below_range])
    above = len([g for g in glucose_list if g > above_range])
    in_range = len([g for g in glucose_list if below_range <= g <= above_range])

    return pd.Series({
        'Time in Range (%)': (in_range / total) * 100 if total else 0,
        'Time in Tight Range (%)': (tight / total) * 100 if total else 0,
        'Time Below Range (%)': (below / total) * 100 if total else 0,
        'Time Above Range (%)': (above / total) * 100 if total else 0
    })

# --- STEP 1: Calculate Learned Metrics ---
glucose_events_df = pd.read_excel(xls, sheet_name='GlucoseEvents')
glucose_events_df.columns = glucose_events_df.columns.str.strip()
glucose_events_df['ParticipantID'] = glucose_events_df['Source File'].str.replace(r'(_MealTagCatSim)?\.xlsx$', '', regex=True)

def parse_glucose(x):
    if isinstance(x, str):
        fixed = re.sub(r'(?<=\d)\s+(?=\d)', ', ', x.strip())
        try:
            values = ast.literal_eval(fixed)
            return [float(val) for val in values if isinstance(val, (int, float, str)) and str(val).replace('.', '', 1).isdigit()]
        except:
            return []
    return []

glucose_events_df['GlucoseLevels'] = glucose_events_df['GlucoseLevels'].apply(parse_glucose)

learned_metrics = glucose_events_df.groupby('ParticipantID')['GlucoseLevels'].apply(
    lambda s: calculate_ranges([g for sublist in s for g in sublist])
)

# Renaming 'Learned' columns to 'Average Learned Response' and adding them to the learned metrics
learned_metrics = learned_metrics.rename(lambda name: f"Average Learned {name}" if 'Learned' in name else name).reset_index()

# --- STEP 2: Process Simulation Sheets ---
dfs = []

for measure, sheet_name in assessment_measures.items():
    try:
        if sheet_name not in xls.sheet_names:
            print(f"⚠️ Sheet {sheet_name} not found.")
            continue

        df = pd.read_excel(xls, sheet_name=sheet_name)
        df.columns = df.columns.str.strip()

        if 'Simulated Glucose Response' not in df.columns:
            print(f"⚠️ Missing 'Simulated Glucose Response' in {sheet_name}")
            continue

        df['Simulated Glucose Response'] = df['Simulated Glucose Response'].apply(
            lambda x: ast.literal_eval(str(x)) if isinstance(x, str) else []
        )

        sim_ranges = df['Simulated Glucose Response'].apply(
            lambda x: calculate_ranges(x).rename(lambda name: f"Simulated {name}")
        )
        df = pd.concat([df, sim_ranges], axis=1)

        df['ParticipantID'] = df['Source File'].str.replace(r'(_MealTagCatSim)?\.xlsx$', '', regex=True)
        df['AssessmentMeasure'] = measure

        # Merging the learned metrics into the simulation data
        df = df.merge(learned_metrics, on='ParticipantID', how='left')
        dfs.append(df)

    except Exception as e:
        print(f"❌ Error processing {sheet_name}: {e}")

# --- STEP 3: Combine All ---
df_all = pd.concat(dfs, ignore_index=True) if dfs else pd.DataFrame()

if not df_all.empty:
    df_all.to_excel('Combined_Cleaned_Data.xlsx', index=False)

    # Specify the desired columns for the summary output
    summary_cols = [
        'ParticipantID', 'AssessmentMeasure',
        'Simulated Time in Range (%)', 'Simulated Time in Tight Range (%)',
        'Simulated Time Below Range (%)', 'Simulated Time Above Range (%)',
        'Average Learned Time in Range (%)', 'Average Learned Time in Tight Range (%)',
        'Average Learned Time Below Range (%)', 'Average Learned Time Above Range (%)'
    ]

    # Selecting only the relevant columns and dropping any NaN values
    summary = df_all[[col for col in summary_cols if col in df_all.columns]].dropna()

    # Create a pivot table with average values for each participant and measure
    summary_pivot = summary.pivot_table(
        index='ParticipantID',
        columns='AssessmentMeasure',
        values=[col for col in summary.columns if col not in ['ParticipantID', 'AssessmentMeasure']],
        aggfunc='mean'
    )

    # Saving the result to an Excel file
    summary_pivot.to_excel('TIR_summary_by_participant_and_measure.xlsx')
    print("✅ Summary written to TIR_summary_by_participant_and_measure.xlsx")
else:
    print("⚠️ No valid data was processed.")


✅ Summary written to TIR_summary_by_participant_and_measure.xlsx


In [ ]:
import pandas as pd
import scipy.stats as stats
import seaborn as sns
import matplotlib.pyplot as plt

# Load the Excel file
file_path = '/content/TIR_summary_by_participant_and_measure.xlsx'
xls = pd.ExcelFile(file_path)

# Load the sheet using the first two rows as a multi-level header
df = xls.parse('Sheet1', header=[0, 1])

# Inspect the actual column names (you've already done this)
print(df.columns.tolist())

#print(df.head())

# Set the correct participant identifier as the index
df.set_index(('Unnamed: 0_level_0', 'AssessmentMeasure'), inplace=True)

# Flatten the multi-index columns
df.columns = [f"{col[0]} {col[1]}".strip() if col[0] else col[1] for col in df.columns]

# Drop unnamed columns
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]


# Descriptive statistics
desc_stats = df.describe()

# Correlation matrix
correlation_matrix = df.corr(numeric_only=True)

# Visualize the correlation matrix with a heatmap
# plt.figure(figsize=(12, 8))
# sns.heatmap(correlation_matrix, annot=True, fmt=".2f", cmap='coolwarm', cbar=True, linewidths=0.5)
# plt.title('Correlation Matrix')
# plt.show()

# Define the simulation types for comparison
simulation_types = [
    "Meal Category Cluster Simulation",
    "Meal Category Cluster Pump  Simulation",
    "Meal Tag Cluster Simulation",
    "Meal Tag and Cluster Simulation"
]

# Define the metrics to test
columns_to_test = [
    "Time Below Range (%)",
    "Time In Target Range (%)",
    "Time In Tight Range (%)",
    "Time Above Range (%)"
]

# Perform paired t-tests for each metric and simulation type
t_tests = {}

for sim in simulation_types:
    t_tests[sim] = {}
    for col in columns_to_test:
        baseline_col = f"Baseline {col}"
        sim_col = f"{sim} {col}"

        if baseline_col in df.columns and sim_col in df.columns:
            t_stat, p_val = stats.ttest_rel(df[baseline_col], df[sim_col])
            t_tests[sim][col] = {"t-statistic": t_stat, "p-value": p_val}
        else:
            t_tests[sim][col] = "Columns not found"

# Output results
print("\nDescriptive Statistics:\n", desc_stats)
print("\nCorrelation Matrix:\n", correlation_matrix)
print("\nPaired T-tests between Baseline and Simulations:\n")
for sim_type, results in t_tests.items():
    print(f"\nSimulation: {sim_type}")
    for metric, res in results.items():
        print(f"  {metric}: {res}")



[('Unnamed: 0_level_0', 'AssessmentMeasure'), ('Simulated Time Above Range (%)', 'Best_Cat_Cluster_Pump'), ('Simulated Time Above Range (%)', 'Best_MealCat_ClusteSim'), ('Simulated Time Above Range (%)', 'best_MealTagSim'), ('Simulated Time Below Range (%)', 'Best_Cat_Cluster_Pump'), ('Simulated Time Below Range (%)', 'Best_MealCat_ClusteSim'), ('Simulated Time Below Range (%)', 'best_MealTagSim'), ('Simulated Time in Range (%)', 'Best_Cat_Cluster_Pump'), ('Simulated Time in Range (%)', 'Best_MealCat_ClusteSim'), ('Simulated Time in Range (%)', 'best_MealTagSim'), ('Simulated Time in Tight Range (%)', 'Best_Cat_Cluster_Pump'), ('Simulated Time in Tight Range (%)', 'Best_MealCat_ClusteSim'), ('Simulated Time in Tight Range (%)', 'best_MealTagSim')]
  Unnamed: 0_level_0 Simulated Time Above Range (%)                         \
   AssessmentMeasure          Best_Cat_Cluster_Pump Best_MealCat_ClusteSim   
0      ParticipantID                            NaN                    NaN   
1   UoM2

In [ ]:
# Reset the index so that 'Participant' becomes a column
df.reset_index(inplace=True)

# Print the column names to inspect them
print(df.columns)

# Flatten columns if they have multiple levels
df.columns = [col[1] if isinstance(col, tuple) else col for col in df.columns]

# Directory to save plots
save_dir = 'plots/'
os.makedirs(save_dir, exist_ok=True)

# Identify columns related to metrics
metrics = [
    "Time Below Range (%)",
    "Time In Target Range (%)",
    "Time In Tight Range (%)",
    "Time Above Range (%)"
]

simulation_types = [
    "Meal Category Cluster Simulation",
    "Meal Category Cluster Pump Simulation",
    "Meal Tag Cluster Simulation",
    "Meal Tag and Cluster Simulation"
]

# Helper function to tidy and plot the data
def tidy_and_plot(metric_name, columns):
    tidy_df = df[['Participant'] + list(columns)].melt(id_vars='Participant',
                                                        var_name='Method',
                                                        value_name=metric_name)

    plt.figure(figsize=(14, 6))
    ax = sns.barplot(x='Participant', y=metric_name, hue='Method', data=tidy_df, palette='Greys')
    plt.title(f'{metric_name} by Participant and Method')
    plt.xlabel('Participant', fontweight='bold')
    plt.ylabel(metric_name, fontweight='bold')
    plt.xticks(rotation=45)
    sns.despine()

    # Move legend outside the plot to the right
    plt.legend(title='Method', bbox_to_anchor=(1.01, 1), loc='upper left', borderaxespad=0)

    plt.tight_layout()  # Ensure everything fits
    plot_filename = f"{metric_name.lower().replace(' ', '_')}_by_participant_and_method.png"
    plt.savefig(os.path.join(save_dir, plot_filename), dpi=300, bbox_inches='tight')  # Ensure the legend is not cut off
    plt.close()


# Prepare the columns for each metric type
metric_groups = {
    'Time Below Range': df.filter(like="Time Below Range").columns,
    'Time In Target Range': df.filter(like="Time In Target Range").columns,
    'Time In Tight Range': df.filter(like="Time In Tight Range").columns,
    'Time Above Range': df.filter(like="Time Above Range").columns
}

# Generate plots for each metric group
for metric_name, columns in metric_groups.items():
    tidy_and_plot(metric_name, columns)

print("✅ Plots saved in the 'plots/' directory.")

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import ast
import os
import numpy as np
from scipy.ndimage import gaussian_filter1d

# Parameters
smooth_sigma = 1.5  # Standard deviation for Gaussian smoothing
participant_id = 'UoM2313'

# Define simulation sheets and file path
file_path = '/content/merged_data.xlsx'
simulation_sheets = [
    'best_MealTagSim',
    'Best_MealCat_ClusteSim',
    'bestMealTagCatResults',
    'Best_Cat_Cluster_Pump'
]

# Load each sheet into a dictionary
xls = pd.ExcelFile(file_path)
sim_data = {sheet: xls.parse(sheet) for sheet in simulation_sheets}

# Plotting function for individual responses
def plot_learned_vs_simulated(df, method_name, output_dir='plots'):
    df_filtered = df[df['Source File'].str.contains(participant_id)]

    if 'Meal Category' in df.columns:
        meal_col = 'Meal Category'
    elif 'MealTag' in df.columns:
        meal_col = 'MealTag'
    else:
        meal_col = None

    cluster_col = 'Cluster' if 'Cluster' in df.columns else None

    os.makedirs(output_dir, exist_ok=True)

    for _, row in df_filtered.iterrows():
        meal_category = row[meal_col] if meal_col else 'Unknown'
        cluster = row[cluster_col] if cluster_col else 'Unknown'

        try:
            sim_response = ast.literal_eval(str(row['Simulated Glucose Response']))
            learned_response = ast.literal_eval(str(row['Learned Glucose Response']))
        except Exception as e:
            print(f"Skipping row due to parse error: {e}")
            continue

        sim_response_smoothed = gaussian_filter1d(sim_response, sigma=smooth_sigma)
        learned_response_smoothed = gaussian_filter1d(learned_response, sigma=smooth_sigma)

        x_sim = [i * 5 for i in range(len(sim_response_smoothed))]  # X in minutes
        x_learned = [i * 5 for i in range(len(learned_response_smoothed))]

        plt.figure(figsize=(10, 6))
        plt.plot(x_sim, sim_response_smoothed, label='Simulated', linestyle='-', color='black')
        plt.plot(x_learned, learned_response_smoothed, label='Learned', linestyle='--', color='black')

        # Add target range lines
        plt.axhline(3.9, color='red', linestyle=':', linewidth=1.5, label='Target Min (3.9 mmol/L)')
        plt.axhline(10.0, color='red', linestyle=':', linewidth=1.5, label='Target Max (10 mmol/L)')

        # title = f'{method_name}'
        # if meal_col:
        #     title += f' - {meal_category}'
        # if cluster_col:
        #     title += f' (Cluster {cluster})'

        # plt.title(title)
        plt.xlabel('Time (minutes)')
        plt.ylabel('Glucose Level (mmol/L)')
        plt.legend()
        plt.grid(True)
        plt.tight_layout()

        safe_meal_category = meal_category.replace(' ', '_').replace('/', '_').replace('\\', '_')
        safe_cluster = str(cluster).replace(' ', '_').replace('/', '_').replace('\\', '_')

        filename = f"{participant_id}_{method_name}_{safe_meal_category}_{safe_cluster}.png".replace(" ", "_")
        filepath = os.path.join(output_dir, filename)
        plt.savefig(filepath, dpi=300)
        print(f"Plot saved to: {filepath}")

        plt.close()


# Aggregated plotting function
def aggregate_and_plot_responses(df, method_name, output_dir='plots'):
    df_filtered = df[df['Source File'].str.contains(participant_id)]

    sim_responses = []
    learned_responses = []

    for _, row in df_filtered.iterrows():
        try:
            sim_response = ast.literal_eval(str(row['Simulated Glucose Response']))
            learned_response = ast.literal_eval(str(row['Learned Glucose Response']))
            if isinstance(sim_response, list) and isinstance(learned_response, list):
                sim_responses.append(sim_response)
                learned_responses.append(learned_response)
        except Exception as e:
            print(f"Skipping row due to parse error: {e}")

    lengths = [len(x) for x in sim_responses]
    if not lengths:
        print(f"No valid responses for {method_name}")
        return

    most_common_length = max(set(lengths), key=lengths.count)
    sim_filtered = [x for x in sim_responses if len(x) == most_common_length]
    learned_filtered = [x for x in learned_responses if len(x) == most_common_length]

    if not sim_filtered or not learned_filtered:
        print(f"No consistent-length data to plot for {method_name}")
        return

    sim_array = np.array(sim_filtered)
    learned_array = np.array(learned_filtered)

    sim_mean = sim_array.mean(axis=0)
    sim_std = sim_array.std(axis=0)

    learned_mean = learned_array.mean(axis=0)
    learned_std = learned_array.std(axis=0)

    sim_mean_smoothed = gaussian_filter1d(sim_mean, sigma=smooth_sigma)
    sim_std_smoothed = gaussian_filter1d(sim_std, sigma=smooth_sigma)

    learned_mean_smoothed = gaussian_filter1d(learned_mean, sigma=smooth_sigma)
    learned_std_smoothed = gaussian_filter1d(learned_std, sigma=smooth_sigma)

    x = np.arange(most_common_length) * 5  # Time in minutes

    plt.figure(figsize=(10, 6))

    # Simulated mean ± SD
    plt.plot(x, sim_mean_smoothed, label='Simulated Mean', linestyle='-', color='black')
    plt.fill_between(x,
                     sim_mean_smoothed - sim_std_smoothed,
                     sim_mean_smoothed + sim_std_smoothed,
                     color='lightgray', hatch='///', edgecolor='black', linewidth=0.5,
                     label='Simulated ±1 SD')

    # Learned mean ± SD
    plt.plot(x, learned_mean_smoothed, label='Learned Mean', linestyle='--', color='black')
    plt.fill_between(x,
                     learned_mean_smoothed - learned_std_smoothed,
                     learned_mean_smoothed + learned_std_smoothed,
                     color='none', hatch='\\\\\\', edgecolor='black', linewidth=0.5,
                     label='Learned ±1 SD')

    # Add target range lines
    plt.axhline(3.9, color='red', linestyle=':', linewidth=1.5, label='Target Min (3.9 mmol/L)')
    plt.axhline(10.0, color='red', linestyle=':', linewidth=1.5, label='Target Max (10 mmol/L)')

    # plt.title(f'Average Glucose Responses - {method_name}')
    plt.xlabel('Time (minutes)')
    plt.ylabel('Glucose Level (mmol/L)')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()

    os.makedirs(output_dir, exist_ok=True)
    filename = f"{participant_id}_{method_name}.png".replace(" ", "_")
    filepath = os.path.join(output_dir, filename)
    plt.savefig(filepath, dpi=300)
    print(f"Plot saved to: {filepath}")

    plt.close()

# Run for each method
for method, df in sim_data.items():
    print(f'\n--- Plotting for method: {method} ---')
    print(f"Columns in '{method}': {df.columns.tolist()}")
    plot_learned_vs_simulated(df, method)

for method, df in sim_data.items():
    print(f'\n--- Aggregated Plot for method: {method} ---')
    aggregate_and_plot_responses(df, method)


In [ ]:
import os

def aggregate_and_plot_responses(df, method_name, output_dir='plots'):
    df_filtered = df[df['Source File'].str.contains(participant_id)]

    sim_responses = []
    learned_responses = []

    for _, row in df_filtered.iterrows():
        try:
            sim_response = ast.literal_eval(str(row['Simulated Glucose Response']))
            learned_response = ast.literal_eval(str(row['Learned Glucose Response']))

            if isinstance(sim_response, list) and isinstance(learned_response, list):
                sim_responses.append(sim_response)
                learned_responses.append(learned_response)
        except Exception as e:
            print(f"Skipping row due to parse error: {e}")

    # Find the most common response length
    lengths = [len(x) for x in sim_responses]
    if not lengths:
        print(f"No valid responses for {method_name}")
        return

    most_common_length = max(set(lengths), key=lengths.count)

    # Filter to only responses of this common length
    sim_filtered = [x for x in sim_responses if len(x) == most_common_length]
    learned_filtered = [x for x in learned_responses if len(x) == most_common_length]

    if not sim_filtered or not learned_filtered:
        print(f"No consistent-length data to plot for {method_name}")
        return

    # Convert to NumPy arrays
    sim_array = np.array(sim_filtered)
    learned_array = np.array(learned_filtered)

    # Compute mean and std
    sim_mean = sim_array.mean(axis=0)
    sim_std = sim_array.std(axis=0)

    learned_mean = learned_array.mean(axis=0)
    learned_std = learned_array.std(axis=0)

    # Apply Gaussian smoothing
    sim_mean_smoothed = gaussian_filter1d(sim_mean, sigma=smooth_sigma)
    sim_std_smoothed = gaussian_filter1d(sim_std, sigma=smooth_sigma)

    learned_mean_smoothed = gaussian_filter1d(learned_mean, sigma=smooth_sigma)
    learned_std_smoothed = gaussian_filter1d(learned_std, sigma=smooth_sigma)

    # Plot
    x = np.arange(most_common_length)
    plt.figure(figsize=(10, 6))

    # Simulated
    plt.plot(x, sim_mean_smoothed, label='Simulated Mean', color='blue')
    plt.fill_between(x,
                     sim_mean_smoothed - sim_std_smoothed,
                     sim_mean_smoothed + sim_std_smoothed,
                     color='blue', alpha=0.2, label='Simulated ±1 SD')

    # Learned
    plt.plot(x, learned_mean_smoothed, label='Learned Mean', color='orange')
    plt.fill_between(x,
                     learned_mean_smoothed - learned_std_smoothed,
                     learned_mean_smoothed + learned_std_smoothed,
                     color='orange', alpha=0.2, label='Learned ±1 SD')

    plt.title(f'Average Glucose Responses - {method_name}')
    plt.xlabel('Time Points')
    plt.ylabel('Glucose Level (mmol/L)')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()

    # Save plot
    os.makedirs(output_dir, exist_ok=True)
    filename = f"{participant_id}_{method_name}.png".replace(" ", "_")
    filepath = os.path.join(output_dir, filename)
    plt.savefig(filepath, dpi=300)
    print(f"Plot saved to: {filepath}")

    plt.close()  # Don't display the plot, just save it


# Run for each method
for method, df in sim_data.items():
    print(f'\n--- Aggregated Plot for method: {method} ---')
    aggregate_and_plot_responses(df, method)


#########

Baseline Meal Catgeorgy stats



########

In [58]:
import pandas as pd
import ast

file_path = '/content/merged_data_July2025.xlsx'

# ✅ Load Excel file and parse the correct sheet
xls = pd.ExcelFile(file_path)
BaselineMealTagCat_df = xls.parse('GlucoseEvents')
BaselineMealTagCat_df.columns = BaselineMealTagCat_df.columns.str.strip()

# ✅ Extract participant ID
BaselineMealTagCat_df['Participant'] = BaselineMealTagCat_df['Source File'].str.extract(r'(UoM\d{4})')
BaselineMealTagCat_df['Source File'] = BaselineMealTagCat_df['Participant']

# ✅ Set participant order
participant_order = [
    'UoM2301', 'UoM2302', 'UoM2304', 'UoM2305', 'UoM2306',
    'UoM2307', 'UoM2308', 'UoM2309', 'UoM2313', 'UoM2314',
    'UoM2320', 'UoM2403', 'UoM2404', 'UoM2405'
]
BaselineMealTagCat_df['Participant'] = pd.Categorical(BaselineMealTagCat_df['Participant'], categories=participant_order, ordered=True)
BaselineMealTagCat_df['Source File'] = pd.Categorical(BaselineMealTagCat_df['Source File'], categories=participant_order, ordered=True)

# ✅ Glucose parser
def parse_glucose_response(x):
    try:
        if isinstance(x, str):
            # Remove square brackets and split by whitespace
            x = x.strip('[]').strip()
            if x:
                return [float(i) for i in x.split()]
        elif isinstance(x, list):
            return [float(i) for i in x]
    except Exception:
        return []
    return []


# ✅ Apply parser and debug
BaselineMealTagCat_df['Baseline Glucose'] = BaselineMealTagCat_df['NetGlucose'].apply(parse_glucose_response)
print(BaselineMealTagCat_df[['NetGlucose', 'Baseline Glucose']].head())  # optional debug

# ✅ Range calculation
tight_range_min = 3.9
tight_range_max = 7.8
below_range = 3.9
above_range = 10.0

def calculate_ranges(glucose_list):
    glucose_list = [g for g in glucose_list if isinstance(g, (int, float))]
    total = len(glucose_list)
    if total == 0:
        return pd.Series({
            'Percentage Time in Target Range': pd.NA,
            'Percentage Time in Tight Range': pd.NA,
            'Percentage Time Below Range': pd.NA,
            'Percentage Time Above Range': pd.NA
        })
    tight = sum(tight_range_min <= g <= tight_range_max for g in glucose_list)
    below = sum(g < below_range for g in glucose_list)
    above = sum(g > above_range for g in glucose_list)
    in_range = sum(below_range <= g <= above_range for g in glucose_list)
    return pd.Series({
        'Percentage Time in Target Range': (in_range / total) * 100,
        'Percentage Time in Tight Range': (tight / total) * 100,
        'Percentage Time Below Range': (below / total) * 100,
        'Percentage Time Above Range': (above / total) * 100
    })

# ✅ Apply range metrics
range_metrics_df = BaselineMealTagCat_df['Baseline Glucose'].apply(calculate_ranges)
BaselineMealTagCat_df = pd.concat([BaselineMealTagCat_df, range_metrics_df], axis=1)

# ✅ Drop rows where parsing failed
BaselineMealTagCat_df = BaselineMealTagCat_df.dropna(subset=['Percentage Time in Target Range'])

# ✅ Group by and average
baseline_result = (
    BaselineMealTagCat_df
    .groupby(['Source File', 'MealType', 'MealTag'], observed=True)[
        ['Percentage Time in Target Range',
         'Percentage Time in Tight Range',
         'Percentage Time Below Range',
         'Percentage Time Above Range']]
    .mean()
    .reset_index()
    .sort_values(['Source File', 'MealType', 'MealTag'])
)

print(baseline_result)




                                          NetGlucose  \
0  [ 7.8         9.44658365 11.63053752 12.680112...   
1  [ 5.6         6.77329182 10.16526876 12.840056...   
2  [ 6.9         9.98550713 13.00948022 14.930065...   
3  [ 4.9         6.12443061  8.98842293  9.380018...   
4  [ 8.3         9.62443061 12.38842293 13.980018...   

                                    Baseline Glucose  
0  [7.8, 9.44658365, 11.63053752, 12.68011295, 12...  
1  [5.6, 6.77329182, 10.16526876, 12.84005647, 14...  
2  [6.9, 9.98550713, 13.00948022, 14.93006588, 14...  
3  [4.9, 6.12443061, 8.98842293, 9.38001885, 8.58...  
4  [8.3, 9.62443061, 12.38842293, 13.98001885, 14...  
     Source File   MealType                  MealTag  \
0        UoM2301  Breakfast               Coffee(x3)   
1        UoM2301  Breakfast          Porridge+Coffee   
2        UoM2301     Dinner  CheesyChickenEnchiladas   
3        UoM2301     Dinner                   Nandos   
4        UoM2301     Dinner              NotReported 

In [68]:
# Define participant groups
group_A_ids = ['UoM2302', 'UoM2304', 'UoM2306', 'UoM2308', 'UoM2320', 'UoM2404']
group_B_ids = ['UoM2301', 'UoM2305', 'UoM2307', 'UoM2309', 'UoM2313', 'UoM2314', 'UoM2403', 'UoM2405']

# Step 1: Map participants to groups
def assign_group(participant):
    if participant in group_A_ids:
        return 'Group A'
    elif participant in group_B_ids:
        return 'Group B'
    else:
        return 'Other'  # in case there's a participant outside these groups

BaselineMealTagCat_df['Group'] = BaselineMealTagCat_df['Participant'].apply(assign_group)

# Step 2: Filter only Group A and B participants (optional)
df_filtered = BaselineMealTagCat_df[BaselineMealTagCat_df['Group'].isin(['Group A', 'Group B'])]

# Step 3: Define a function to calculate IQR
def iqr(x):
    return np.percentile(x, 75) - np.percentile(x, 25)

# Step 4: Group by Group and MealType and calculate stats
summary_stats = (
    df_filtered
    .groupby(['Group', 'MealType'])
    [['Percentage Time in Target Range',
      'Percentage Time in Tight Range',
      'Percentage Time Below Range',
      'Percentage Time Above Range']]
    .agg(['mean', 'std', 'median', iqr])
    .reset_index()
)

# Define the meal order and filter for those meals only
meal_order = ['Breakfast', 'Lunch', 'Dinner', 'Snack']

# Step 1: Make sure MealType is categorical with the specific order
summary_stats['MealType'] = pd.Categorical(
    summary_stats['MealType'],
    categories=meal_order,
    ordered=True
)

# Step 2: Filter only these meals
summary_stats_filtered = summary_stats[summary_stats['MealType'].isin(meal_order)]

# Step 3: Sort by Group and MealType according to the specified order
summary_stats_filtered = summary_stats_filtered.sort_values(['Group', 'MealType'])

print(summary_stats_filtered)



# Export to Excel
# Flatten MultiIndex columns
summary_stats_filtered.columns = [' '.join(col).strip() if isinstance(col, tuple) else col
                                  for col in summary_stats_filtered.columns]

# Export to Excel
output_file = 'group_meal_summary_stats.xlsx'
summary_stats_filtered.to_excel(output_file, index=False)

print(f"Summary statistics exported to {output_file}")



      Group   MealType Percentage Time in Target Range                        \
                                                  mean        std     median   
0   Group A  Breakfast                       83.692529  16.017660  87.500000   
3   Group A      Lunch                       82.359279  17.852944  87.500000   
2   Group A     Dinner                       79.004854  22.748955  87.500000   
4   Group A      Snack                       85.153654  18.234780  91.666667   
5   Group B  Breakfast                       62.343945  21.957811  64.583333   
10  Group B      Lunch                       58.307940  24.575711  60.416667   
8   Group B     Dinner                       57.186214  28.884274  62.500000   
14  Group B      Snack                       58.266773  26.492053  60.416667   

                                                           ...  \
          iqr       mean        std     median        iqr  ...   
0   22.916667  64.643199  23.892775  64.583333  39.583333  ...   
3

/tmp/ipython-input-68-2545187380.py:32: PerformanceWarning: indexing past lexsort depth may impact performance.
  .reset_index()


##########


Meal Tag Results


##########

In [60]:
import pandas as pd
import ast

# Load the correct sheet
MealTagCat_df = xls.parse('bestMealTagCatResults')
MealTagCat_df.columns = MealTagCat_df.columns.str.strip()

# Extract participant ID
MealTagCat_df['Participant'] = MealTagCat_df['Source File'].str.extract(r'(UoM\d{4})')
MealTagCat_df['Source File'] = MealTagCat_df['Participant']

# Sort participants
participant_order = [
    'UoM2301', 'UoM2302', 'UoM2304', 'UoM2305', 'UoM2306',
    'UoM2307', 'UoM2308', 'UoM2309', 'UoM2313', 'UoM2314',
    'UoM2320', 'UoM2403', 'UoM2404', 'UoM2405'
]
MealTagCat_df['Participant'] = pd.Categorical(MealTagCat_df['Participant'], categories=participant_order, ordered=True)
MealTagCat_df['Source File'] = pd.Categorical(MealTagCat_df['Source File'], categories=participant_order, ordered=True)

# Robust glucose parser
def parse_glucose_response(x):
    try:
        if isinstance(x, list):
            return [float(i) for i in x]
        elif isinstance(x, str):
            parsed = ast.literal_eval(x)
            if isinstance(parsed, list):
                return [float(i) for i in parsed]
            elif isinstance(parsed, str):  # handle nested stringified list
                parsed2 = ast.literal_eval(parsed)
                if isinstance(parsed2, list):
                    return [float(i) for i in parsed2]
    except Exception as e:
        return []
    return []

MealTagCat_df['Parsed Glucose'] = MealTagCat_df['Simulated Glucose Response'].apply(parse_glucose_response)

# Thresholds
tight_range_min = 3.9
tight_range_max = 7.8
below_range = 3.9
above_range = 10.0

# Calculate percentage time in ranges
def calculate_ranges(glucose_list):
    glucose_list = [g for g in glucose_list if isinstance(g, (int, float))]
    total = len(glucose_list)
    if total == 0:
        return pd.Series({
            'Percentage Time in Target Range': pd.NA,
            'Percentage Time in Tight Range': pd.NA,
            'Percentage Time Below Range': pd.NA,
            'Percentage Time Above Range': pd.NA
        })
    tight = sum(tight_range_min <= g <= tight_range_max for g in glucose_list)
    below = sum(g < below_range for g in glucose_list)
    above = sum(g > above_range for g in glucose_list)
    in_range = sum(below_range <= g <= above_range for g in glucose_list)
    return pd.Series({
        'Percentage Time in Target Range': (in_range / total) * 100,
        'Percentage Time in Tight Range': (tight / total) * 100,
        'Percentage Time Below Range': (below / total) * 100,
        'Percentage Time Above Range': (above / total) * 100
    })

# Apply calculation
range_metrics_df = MealTagCat_df['Parsed Glucose'].apply(calculate_ranges)
MealTagCat_df = pd.concat([MealTagCat_df, range_metrics_df], axis=1)

# ✅ Drop rows where parsing failed
MealTagCat_df = MealTagCat_df.dropna(subset=['Percentage Time in Target Range'])

# ✅ Group and average only real values
result = (
    MealTagCat_df
    .groupby(['Source File', 'Meal Category', 'MealTag'], observed=True)[
        ['Percentage Time in Target Range',
         'Percentage Time in Tight Range',
         'Percentage Time Below Range',
         'Percentage Time Above Range']]
    .mean()
    .reset_index()
    .sort_values(['Source File', 'Meal Category', 'MealTag'])
)

# Show or export result
print(result)



     Source File Meal Category                  MealTag  \
0        UoM2301     Breakfast               Coffee(x3)   
1        UoM2301     Breakfast          Porridge+Coffee   
2        UoM2301        Dinner  CheesyChickenEnchiladas   
3        UoM2301        Dinner                   Nandos   
4        UoM2301        Dinner              NotReported   
...          ...           ...                      ...   
1734     UoM2405         Snack              ChocBiscuit   
1735     UoM2405         Snack             ChocBiscuits   
1736     UoM2405         Snack              Coffee+Milk   
1737     UoM2405         Snack              JamieDodger   
1738     UoM2405         Snack            YorkieBiscuit   

      Percentage Time in Target Range  Percentage Time in Tight Range  \
0                          100.000000                       29.166667   
1                           47.916667                        0.000000   
2                           62.500000                       20.833333   

In [61]:
# Function to calculate IQR
def calculate_iqr(series):
    return series.quantile(0.75) - series.quantile(0.25)

# Define aggregation dictionary
agg_funcs = {
    'Percentage Time in Target Range': ['mean', 'std', 'median', calculate_iqr],
    'Percentage Time in Tight Range': ['mean', 'std', 'median', calculate_iqr],
    'Percentage Time Below Range': ['mean', 'std', 'median', calculate_iqr],
    'Percentage Time Above Range': ['mean', 'std', 'median', calculate_iqr]
}

# Group by Participant and Meal Category and compute metrics
summary_stats = (
    MealTagCat_df
    .groupby(['Participant', 'Meal Category'], observed=True)
    .agg(agg_funcs)
    .round(1)
)

# Clean up column names (flatten MultiIndex)
summary_stats.columns = [
    f'{metric} ({stat})'
    for metric, stat in summary_stats.columns
]

# Reset index for a clean DataFrame
summary_stats = summary_stats.reset_index()

# Display result
print(summary_stats)



   Participant Meal Category  Percentage Time in Target Range (mean)  \
0      UoM2301     Breakfast                                    74.0   
1      UoM2301        Dinner                                    67.1   
2      UoM2301         Lunch                                    69.6   
3      UoM2301         Snack                                    79.7   
4      UoM2302     Breakfast                                    84.7   
..         ...           ...                                     ...   
63     UoM2404         Snack                                    69.3   
64     UoM2405     Breakfast                                    73.2   
65     UoM2405        Dinner                                    69.7   
66     UoM2405         Lunch                                    68.0   
67     UoM2405         Snack                                    72.7   

    Percentage Time in Target Range (std)  \
0                                    36.8   
1                                    21.6   


In [71]:
# Function to calculate IQR
def calculate_iqr(series):
    return series.quantile(0.75) - series.quantile(0.25)

# Define participant groups
group_A_ids = ['UoM2302', 'UoM2304', 'UoM2306', 'UoM2308', 'UoM2320', 'UoM2404']
group_B_ids = ['UoM2301', 'UoM2305', 'UoM2307', 'UoM2309', 'UoM2313', 'UoM2314', 'UoM2403', 'UoM2405']

# Assign baseline group based on Participant
def assign_group(participant):
    if participant in group_A_ids:
        return 'Group A'
    elif participant in group_B_ids:
        return 'Group B'
    else:
        return 'Unknown'

MealTagCat_df['Baseline Group'] = MealTagCat_df['Participant'].apply(assign_group)
MealTagCat_df = MealTagCat_df[MealTagCat_df['Baseline Group'] != 'Unknown']

# Filter to only desired meal categories
filtered_df = MealTagCat_df[MealTagCat_df['Meal Category'].isin(['Breakfast', 'Lunch', 'Dinner', 'Snack'])]

# Aggregation dictionary with IQR included
agg_funcs = {
    'Percentage Time in Target Range': ['mean', 'std', 'median', calculate_iqr],
    'Percentage Time in Tight Range': ['mean', 'std', 'median', calculate_iqr],
    'Percentage Time Below Range': ['mean', 'std', 'median', calculate_iqr],
    'Percentage Time Above Range': ['mean', 'std', 'median', calculate_iqr]
}

# Group by Baseline Group and Meal Category
grouped_stats = (
    filtered_df
    .groupby(['Baseline Group', 'Meal Category'], observed=True)
    .agg(agg_funcs)
    .round(1)
)

# Flatten MultiIndex columns
grouped_stats.columns = [
    f'{metric} ({stat})' for metric, stat in grouped_stats.columns
]

grouped_stats = grouped_stats.reset_index()

print(grouped_stats)

# Export to Excel
output_filename = 'meal_analysis_results.xlsx'
grouped_stats.to_excel(output_filename, index=False)

print(f"Data exported successfully to {output_filename}")




  Baseline Group Meal Category  Percentage Time in Target Range (mean)  \
0        Group A     Breakfast                                    84.9   
1        Group A        Dinner                                    73.8   
2        Group A         Lunch                                    79.9   
3        Group A         Snack                                    79.1   
4        Group B     Breakfast                                    60.9   
5        Group B        Dinner                                    54.0   
6        Group B         Lunch                                    56.8   
7        Group B         Snack                                    50.4   

   Percentage Time in Target Range (std)  \
0                                   16.0   
1                                   24.5   
2                                   18.6   
3                                   20.2   
4                                   26.6   
5                                   31.1   
6                        

In [64]:
import pandas as pd

# Your calculate_iqr function (same as before)
def calculate_iqr(series):
    return series.quantile(0.75) - series.quantile(0.25)

# Participant groups (same for both)
group_A_ids = ['UoM2302', 'UoM2304', 'UoM2306', 'UoM2308', 'UoM2320', 'UoM2404']
group_B_ids = ['UoM2301', 'UoM2305', 'UoM2307', 'UoM2309', 'UoM2313', 'UoM2314', 'UoM2403', 'UoM2405']

def assign_group(participant):
    if participant in group_A_ids:
        return 'Group A'
    elif participant in group_B_ids:
        return 'Group B'
    else:
        return 'Unknown'

# Assign groups to MealTagCat_df (assuming MealTagCat_df contains all data)
MealTagCat_df['Baseline Group'] = MealTagCat_df['Participant'].apply(assign_group)
MealTagCat_df = MealTagCat_df[MealTagCat_df['Baseline Group'] != 'Unknown']

# Filter meal categories
filtered_df = MealTagCat_df[MealTagCat_df['Meal Category'].isin(['Breakfast', 'Lunch', 'Dinner', 'Snack'])]

# Aggregation dictionary
agg_funcs = {
    'Percentage Time in Target Range': ['mean', 'std', 'median', calculate_iqr],
    'Percentage Time in Tight Range': ['mean', 'std', 'median', calculate_iqr],
    'Percentage Time Below Range': ['mean', 'std', 'median', calculate_iqr],
    'Percentage Time Above Range': ['mean', 'std', 'median', calculate_iqr]
}

# Let's assume you have separate dataframes or indicators for Baseline vs MealTag data.
# But your code shows the same dataset MealTagCat_df is used for both baseline and meal tag results.

# If baseline results and meal tag results come from the same dataframe but differ by some column,
# you need to filter or split them accordingly.
# For now, I'll assume the same filtered_df applies for both baseline and meal tag,
# just demonstrating how to combine and export.

# Compute baseline stats
baseline_stats = (
    filtered_df
    .groupby(['Baseline Group', 'Meal Category'], observed=True)
    .agg(agg_funcs)
    .round(1)
)

baseline_stats.columns = [f'{metric} ({stat})' for metric, stat in baseline_stats.columns]
baseline_stats = baseline_stats.reset_index()
baseline_stats['Result Type'] = 'Baseline'

# Compute meal tag stats - assuming this differs somehow,
# if you have another dataframe or different filtering logic, apply here.
meal_tag_stats = (
    filtered_df
    .groupby(['Baseline Group', 'Meal Category'], observed=True)
    .agg(agg_funcs)
    .round(1)
)

meal_tag_stats.columns = [f'{metric} ({stat})' for metric, stat in meal_tag_stats.columns]
meal_tag_stats = meal_tag_stats.reset_index()
meal_tag_stats['Result Type'] = 'Meal Tag'

# Combine both
combined_stats = pd.concat([baseline_stats, meal_tag_stats], ignore_index=True)

# Define desired order for Meal Category
meal_order = ['Breakfast', 'Lunch', 'Dinner', 'Snack']

# Convert Meal Category to categorical for sorting
combined_stats['Meal Category'] = pd.Categorical(combined_stats['Meal Category'], categories=meal_order, ordered=True)

# Sort by Baseline Group, Meal Category, and then Result Type
combined_stats = combined_stats.sort_values(by=['Baseline Group', 'Meal Category', 'Result Type'])

# Export to Excel
combined_stats.to_excel('combined_results.xlsx', index=False)

print("Combined results saved to 'combined_results.xlsx'")


Combined results saved to 'combined_results.xlsx'


In [65]:
!pip install xlsxwriter


output_filepath = '/content/MealTag_Cat_analysis_summary.xlsx'  # Adjust path as needed

with pd.ExcelWriter(output_filepath, engine='xlsxwriter') as writer:
    # Save baseline mean per Source File, Meal Category, Grouped
    summary_stats_filtered.to_excel(writer, sheet_name='Grouped_Baseline_Stats', index=False)

    # Save overall mean per Source File, Meal Category, by Participant
    baseline_result.to_excel(writer, sheet_name='Baseline_Stats', index=False)

    # Save overall mean per Source File, Meal Category, MealTag
    result.to_excel(writer, sheet_name='Mean_Per_MealTag', index=False)

    # Save summary stats per Participant and Meal Category (mean, std, median, IQR)
    summary_stats.to_excel(writer, sheet_name='Participant_Meal_Summary', index=False)

    # Save grouped stats per Baseline Group and Meal Category (mean, std, median, IQR)
    grouped_stats.to_excel(writer, sheet_name='Group_Meal_Summary', index=False)

    # Export to Excel
    combined_stats.to_excel(writer, sheet_name='combined_results.xlsx', index=False)

print(f"All outputs saved to {output_filepath}")



NotImplementedError: Writing to Excel with MultiIndex columns and no index ('index'=False) is not yet implemented.